In [16]:
from dotenv import load_dotenv
import os

load_dotenv(verbose=True)
key = os.getenv('OPENAI_API_KEY')

In [17]:
from typing import List, Dict, Annotated
from langchain_teddynote.tools import GoogleNews
from langchain_experimental.utilities import PythonREPL
from langchain.tools import tool

In [18]:
@tool
def search_news(query: str) -> List[Dict[str, str]]:
    """Search Google News by input keyword"""
    news_tool = GoogleNews()
    return news_tool.search_by_keyword(query, k=5)

In [19]:
@tool
def python_repl_tool(
    # 매개변수 이름은 code 
    # str은 매개 변수의 타입.
    # "The python ~~ your chart." 는 매개변수에 대한 설명.
    code: Annotated[str, "The python code to execute to generate your chart."],
):
    """Use this to execute python code. If you want to see the output of a value,
    you should print it out with `print(...)`. This is visible to the user."""
    result = ""
    try:
        result = PythonREPL().run(code)
    except BaseException as e:
        print(f"Failed to execute. Error: {repr(e)}")
    finally:
        return result

In [20]:
print(f"도구 이름: {search_news.name}")
print(f"도구 설명: {search_news.description}")
print(f"도구 이름: {python_repl_tool.name}")
print(f"도구 설명: {python_repl_tool.description}")

도구 이름: search_news
도구 설명: Search Google News by input keyword
도구 이름: python_repl_tool
도구 설명: Use this to execute python code. If you want to see the output of a value,
    you should print it out with `print(...)`. This is visible to the user.


In [21]:
tools = [search_news, python_repl_tool]

### **Agent 프롬프트 생성**

- `chat_history` : 이전 대화 내용을 저장하는 변수
- `agent_scratchpad` : 에이전트가 검색하는 과정들이나 내용 등을 임시로 저장하는 변수
- `input` : 사용자의 입력

In [22]:
from langchain_core.prompts import ChatPromptTemplate

In [8]:
# 프롬프트 생성. 프롬프트는 에이전트에게 모델이 수행할 작업을 설명하는 텍스트를 제공합니다. (도구의 이름과 역할을 입력)
prompt = ChatPromptTemplate.from_messages(
    [
        (                                           
            "system",
            "You are a helpful assistant. "             
            "Make sure to use the `search_news` tool for searching keyword related news.",
        ),
        ("placeholder", "{chat_history}"),          # 이전 대화 내용을 넣을 곳을 잡아둔다.
        ("human", "{input}"),                       # 사용자 입력

        # 에이전트가 검색하는 과정들이나 내용 등을 끄적이는 메모장 같은 공간을 플레이스 홀더로 만들어준다
        ("placeholder", "{agent_scratchpad}"),       
    ]
)

### **Agent 생성**

In [9]:
from langchain_openai import ChatOpenAI
from langchain.agents import create_tool_calling_agent

In [10]:
# LLM 정의
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [11]:
# Agent 생성
agent = create_tool_calling_agent(llm, tools, prompt)

### **AgentExecutor**

`AgentExecutor`는 도구를 사용하는 에이전트를 실행하는 클래스입니다.
<br>

**속성**

- `agent`: 실행 루프의 각 단계에서 계획을 생성하고 행동을 결정하는 에이전트
- `tools`: 에이전트가 사용할 수 있는 유효한 도구 목록
- `return_intermediate_steps`: 최종 출력과 함께 에이전트의 중간 단계 경로를 반환할지 여부
- `max_iterations`: 실행 루프를 종료하기 전 최대 단계 수
- `max_execution_time`: 실행 루프에 소요될 수 있는 최대 시간 
- `early_stopping_method`: 에이전트가 AgentFinish를 반환하지 않을 때 사용할 조기 종료 방법. ("force" or "generate")
    - `"force"` 는 시간 또는 반복 제한에 도달하여 중지되었다는 문자열을 반환합니다.
    - `"generate"` 는 에이전트의 LLM 체인을 마지막으로 한 번 호출하여 이전 단계에 따라 최종 답변을 생성합니다.
- `handle_parsing_errors`: 에이전트의 출력 파서에서 발생한 오류 처리 방법. (True, False, 또는 오류 처리 함수)
- `trim_intermediate_steps`: 중간 단계를 트리밍하는 방법. (-1 trim 하지 않음, 또는 트리밍 함수)

**메소드**

1. `invoke`: 에이전트 실행
2. `stream`: 최종 출력에 도달하는 데 필요한 단계를 스트리밍

In [12]:
from langchain.agents import AgentExecutor

In [13]:
# AgentExecutor 생성
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    max_iterations=10,
    max_execution_time=10,
    handle_parsing_errors=True,
)

In [14]:
# AgentExecutor 실행
result = agent_executor.invoke({"input": "AI 투자와 관련된 뉴스를 검색해 주세요."})



> Entering new AgentExecutor chain...

Invoking: `search_news` with `{'query': 'AI 투자'}`


[{'url': 'https://news.google.com/rss/articles/CBMiVkFVX3lxTE9HMzNkMXZaZ0k2ekVETmtLNVI2bVVrQlBkbVE3Ry1IenhrWkR1d2pCeHk1ZnNIVjVSVWx4VDRTdjZxcWtsdlhLYTJ4aFVzQzkyNHhsa2pB?oc=5', 'content': '"투자·인재·제도 열악한데 AI 3강?"…韓, 마이너리그 전락 \'우려\' - ZD넷 코리아'}, {'url': 'https://news.google.com/rss/articles/CBMiWkFVX3lxTE4wakVoSDRYZkVkc2plSk9seVRQWkxOVnRNcWVObzEzT1ZtU3FPTzJ6SnRYZk5WbEhDUTFacXVPdjFpYzR4SXM3SG9LN2pyTE5xXzk2T2tPX0NCUdIBVEFVX3lxTE1FVHBlQ2hSOHV6NmxPbVFRaWpYd3doN0dEMXA2UmwteWo0a2J2UVloV3d3NUY0YzRkLUpzXzhCMmkzSFF1UGNCejFaeDFrMDJIVWNuRg?oc=5', 'content': '엔비디아가 투자한 AI 스타트업, 430억원 추가 확보 [긱스 플러스] - 한국경제'}, {'url': 'https://news.google.com/rss/articles/CBMiU0FVX3lxTE54ZmNPOXRVVmV3VFVxeExUX0I2Q29FeURuYjczV1hUSzVGb0VlMWJ2Z0g5MlhxdklSR2FVTzhDWEt2R3J3ZnJHR0d0eHVDVi10SnZB?oc=5', 'content': "멀티모달 AI 영상 전문 '트웰브랩스' 3000만달러 전략투자 유치 - 네이트 뉴스"}, {'url': 'https://news.google.com/rss/articles/CBMi2gJBVV95cUxQYzQxQXJTdFNIV

In [15]:
print("Agent 실행 결과:")
print(result["output"])

Agent 실행 결과:
다음은 AI 투자와 관련된 최근 뉴스입니다:

1. **[투자·인재·제도 열악한데 AI 3강?…韓, 마이너리그 전락 '우려'](https://news.google.com/rss/articles/CBMiVkFVX3lxTE9HMzNkMXZaZ0k2ekVETmtLNVI2bVVrQlBkbVE3Ry1IenhrWkR1d2pCeHk1ZnNIVjVSVWx4VDRTdjZxcWtsdlhLYTJ4aFVzQzkyNHhsa2pB?oc=5)** - ZD넷 코리아

2. **[엔비디아가 투자한 AI 스타트업, 430억원 추가 확보](https://news.google.com/rss/articles/CBMiWkFVX3lxTE4wakVoSDRYZkVkc2plSk9seVRQWkxOVnRNcWVObzEzT1ZtU3FPTzJ6SnRYZk5WbEhDUTFacXVPdjFpYzR4SXM3SG9LN2pyTE5xXzk2T2tPX0NCUdIBVEFVX3lxTE1FVHBlQ2hSOHV6NmxPbVFRaWpYd3doN0dEMXA2UmwteWo0a2J2UVloV3d3NUY0YzRkLUpzXzhCMmkzSFF1UGNCejFaeDFrMDJIVWNuRg?oc=5)** - 한국경제

3. **[멀티모달 AI 영상 전문 '트웰브랩스' 3000만달러 전략투자 유치](https://news.google.com/rss/articles/CBMiU0FVX3lxTE54ZmNPOXRVVmV3VFVxeExUX0I2Q29FeURuYjczV1hUSzVGb0VlMWJ2Z0g5MlhxdklSR2FVTzhDWEt2R3J3ZnJHR0d0eHVDVi10SnZB?oc=5)** - 네이트 뉴스

4. **[칼럼 | 이제는 수익을 고민할 때··· 생성형 AI의 비즈니스 가치를 높이는 팁 5가지](https://news.google.com/rss/articles/CBMi2gJBVV95cUxQYzQxQXJTdFNIVkVjYXlNTDVtRHJqUm1hOEl4NWVFSG1hVjZURFJOWFV4elB1dDZ3NVFWSHU1MGc3WnJ